# Grammar of personhood (machine learning)



## Intro

I'm working on my personification chapter now. So far I've measured an ["agency index"](https://twitter.com/quadrismegistus/status/1059305496211931136), trying to find trends in how words gain/lose (syntactic) "agency" over time. This has been interesting, but what's missing is that personifications don't just do things (as the agency index captures), they do *human* things ("let not Ambition *mock*") and have *human* things ("Honour's voice").

So I'm trying to move on from this 'syntax of agency' to a broader 'grammar of personhood'. I took a failed stopover at the 'semantics of personhood' via word2vec: my 'human words' vector didn't end up being that interesting an index for other words, i.e. didn't seem to capture personification effects. Now I'm moving back to the syntactic BookNLP-style data (subject-verb, modifier-noun, etc) I've collected about the nouns for the Chadwyck Healey poetry collections.

I'm wondering whether a classifier to separate human vs. non-human (maybe human vs. object) words by way of the distribution of other words ('collocated' by syntax): and then use that classifier to estimate the 'humanness' of all words, not just those in the cross-validation experiment? I did this in a smaller related project on animal stories, and according to the cross-validation results, the model found it easier to separate humans and animals in novels than it did in these anthropomorphic animal stories, which seemed right. But here I want to use a classifier to estimate the humanness of all, even non-human/object words like abstract nouns, to see if that changes over time?

## Decide an initial set of words

In [1]:
# Load words from the project-wide 25K
from lit.tools import read_ld
all_words = {d['word'] for d in read_ld('data.worddb.txt')}
len(all_words),list(all_words)[:10]

>> streaming as tsv: data.worddb.txt
   done [0.5 seconds]


(25000,
 [u'fawn',
  u'nunnery',
  u'woods',
  u'spiders',
  u'hanging',
  u'woody',
  u'disobeying',
  u'canes',
  u'scold',
  u'originality'])

## Transform slingshot results into booknlp-like data
Code adapted from the classification work in the [Wild Animal Stories notebook](http://localhost:8888/lab/tree/workspace%2Fwildanimalstories%2Fexperiments.ipynb).

In [2]:
import os,pandas as pd,numpy as np,itertools
from lit import tools
rels = {
        'poss':'Possessive',
        'nsubj':'Subject',
        'nsubjpass':'Subject (passive)',
        'dobj':'Object',
        'amod':'Modifier',
        'compound':'Modifier',
        'appos':'Modifier',
        'attr':'Modifier',
        'dative':'Object'
       }

PATH_TO_SLINGSHOT_RESULT_DATA = '../syntax/results_slingshot/spacy_syntax/parse_path2/cache/'

In [3]:
def transform_results(fn=PATH_TO_SLINGSHOT_RESULT_DATA):
    import pandas as pd,os
    from mpi_slingshot import stream_results
    for path,data in stream_results(fn):
        if '.ipynb' in path: continue
        sent_ld=[]
        num_sent=0
        fn=os.path.split(path)[-1]
        for dx in data:
            if sent_ld and dx['sent_start']!=sent_ld[-1]['sent_start']:
                old=get_booknlp_like_data(sent_ld)
                num_sent+=1
                for odx in old:
                    odx['num_sent']=num_sent
                    odx['fn']=fn
                    yield odx
                    sent_ld=[]
            sent_ld+=[dx]

In [4]:
def get_booknlp_like_data(sent_ld,pos_only={'NOUN'},lemma=False):
    """
    Modifiers
    Nouns possessed by characters: poss
    Adjectives modifying characters: 
    Verbs of which character is a subject
    Verbs of which character is an object
    
    rels = {'poss':'Possessive',
           'nsubj':'Subject',
           'dobj':'Object',
           'amod':'Modifier'}
    """
    
    old=[]
    for dx in sent_ld:
        word=dx['lemma'] if lemma else dx['word']
        rel=dx['dep']
        head=dx['head_lemma'] if lemma else dx['head']
        pos=dx['pos']
        word,head=word.lower(),head.lower()
        if not word in all_words or not pos in pos_only: continue
        word_dx={'head':head,'word':word,'rel':rel}
        old+=[word_dx]
    return old

#### Ran this on Sherlock:

In [5]:
# Create iterator
transformer = transform_results(PATH_TO_SLINGSHOT_RESULT_DATA)
#pd.DataFrame(list(itertools.islice(transformer,10)))

In [6]:
#tools.writegen('./data.booknlp_like_data.chadwyck_poetry.txt', transform_results)
# last run: 2/3/2019 13:49 PST

#### Downloaded this data (*data.booknlp_like_data.chadwyck_poetry.txt.gz*) to *data_booknlp/*

Data appears as:

| fn             | head     | num_sent | rel      | word      |
|----------------|----------|----------|----------|-----------|
| Z400605772.xml | are      | 1        | nsubj    | hills     |
| Z400605772.xml | knows    | 2        | dobj     | roads     |
| Z400605772.xml | knows    | 2        | conj     | moves     |
| Z400605772.xml | in       | 3        | pobj     | circles   |
| Z400605772.xml | within   | 3        | pobj     | head      |
| Z400605772.xml | has      | 4        | dobj     | say       |
| Z400605772.xml | is       | 5        | nsubj    | river     |
| Z400605772.xml | lie      | 5        | nsubj    | winds     |
| Z400605772.xml | at       | 6        | pobj     | dawn      |
| Z400605772.xml | sees     | 6        | dobj     | skies     |
| Z400605772.xml | feels    | 7        | nsubj    | shadows   |
| Z400605772.xml | of       | 7        | pobj     | night     |
| Z400605772.xml | recline  | 7        | dobj     | fingers   |
| Z400605772.xml | on       | 7        | pobj     | eyes      |
| Z400605772.xml | welcomes | 8        | dobj     | sun       |
| Z400605772.xml | sun      | 8        | conj     | rain      |
| Z400605772.xml | has      | 9        | nsubj    | landscape |
| Z400605772.xml | has      | 9        | dobj     | depth     |
| Z400605772.xml | depth    | 9        | conj     | height    |
| Z400605772.xml | city     | 10       | ROOT     | city      |
| Z400605772.xml | burns    | 10       | compound | passion   |
| Z400605772.xml | like     | 10       | pobj     | burns     |
| Z400605772.xml | walks    | 11       | compound | morning   |
| Z400605772.xml | of       | 11       | pobj     | walks     |
| Z400605772.xml | on       | 11       | pobj     | wave      |
| Z400605772.xml | of       | 11       | pobj     | sand      |

## Machine learning human nouns
Code adapted from the classification work in the [Wild Animal Stories notebook](http://localhost:8888/lab/tree/workspace%2Fwildanimalstories%2Fexperiments.ipynb).

#### Decide groups

In [7]:
import lit
CP=lit.load_corpus('ChadwyckPoetry')
CPgroups = CP.new_grouping()
CPgroups.group_by_author_at_30(yearbin=25)
CPgroups.prune_groups(min_group=1600,max_group=2000,min_len=10)
fn2group=dict((k.split('/')[-1]+'.xml',v) for k,v in CPgroups.textid2group.items())
fn2group.items()[:5]

>> reading config files...
>> streaming as tsv: /Users/ryan/DH/lit/corpus/chadwyck_poetry/corpus-metadata.ChadwyckPoetry.txt
   done [3.3 seconds]


[(u'Z200427100.xml', '1850-1874'),
 (u'Z200358033.xml', '1975-1999'),
 (u'Z400369280.xml', '1900-1924'),
 (u'Z300173395.xml', '1850-1874'),
 (u'Z200137391.xml', '1850-1874')]

In [8]:
from lit import tools
import os
def transform_booknlp_like_data(fn='data_booknlp/data.booknlp_like_data.chadwyck_poetry.txt.gz',
                                odir='data_booknlp/data_by_quarter_century/'):
    """
    save booknlp-like data in separate files by group
    """
    if not os.path.exists(odir): os.makedirs(odir)
    group2f={}
    header=None
    for dx in tools.readgen(fn):
        if not header: header=sorted(list(dx.keys()))
        group=fn2group.get(dx['fn'])
        if not group: continue
        dx['group']=group
        ofn=os.path.join(odir,group+'.txt')
        import codecs
        if not group in group2f:
            f=group2f[group]=codecs.open(ofn,'w',encoding='utf-8')
            f.write('\t'.join(h for h in header) + '\n')
        f=group2f[group]
        f.write('\t'.join(dx.get(h,'') for h in header) + '\n')

In [9]:
#transform_booknlp_like_data()

#### Decide fields

In [10]:
FIELDS_WANTED = ['VG.Human','VG.Object','VG.Animal']

In [11]:
from lit.tools.freqs import get_fields
fields=get_fields()
#fields

>> streaming as tsv: /Users/ryan/DH/Dissertation/abstraction/words/data.fields.txt
   done [0.1 seconds]


In [12]:
word2field={}
for field in FIELDS_WANTED:
    for word in fields.get(field,[]):
        word2field[word]=field
word2field.items()[:25]

[(u'peacock', 'VG.Animal'),
 (u'coach', 'VG.Human'),
 (u'liar', 'VG.Human'),
 (u'rabbit', 'VG.Animal'),
 (u'corps', 'VG.Human'),
 (u'fox', 'VG.Animal'),
 (u'bull', 'VG.Animal'),
 (u'dollar', 'VG.Object'),
 (u'commoner', 'VG.Human'),
 (u'obstruction', 'VG.Object'),
 (u'manager', 'VG.Human'),
 (u'pervert', 'VG.Human'),
 (u'gang', 'VG.Human'),
 (u'zinc', 'VG.Object'),
 (u'skin', 'VG.Object'),
 (u'aristocrat', 'VG.Human'),
 (u'chair', 'VG.Object'),
 (u'captain', 'VG.Human'),
 (u'milk', 'VG.Object'),
 (u'equipment', 'VG.Object'),
 (u'voter', 'VG.Human'),
 (u'grape', 'VG.Object'),
 (u'buddy', 'VG.Human'),
 (u'pioneer', 'VG.Human'),
 (u'gymnast', 'VG.Human')]

#### Decide rels

In [13]:
rels = {
    'poss':'Possessive',
    'nsubj':'Subject',
    'dobj':'Object',
    'amod':'Modifier'
}

REL_WORD = True  # True for rel_word ('dobj_roads') or False for word (just 'roads')

#### Within groups, start classifying

In [14]:
def open_group_file(fn='data_booknlp/data_by_quarter_century/1600-1624.txt',only_rels=rels,only_fields=True):
    import pandas as pd
    df=pd.read_csv(fn,sep='\t',encoding='utf-8',quoting=3,error_bad_lines=False)
    
    if only_rels: df=df.loc[df.rel.isin(rels)]
    df['field']=[word2field.get(w,'') for w in df.word]
    if only_fields: df=df.loc[df.field!='']
    return df

In [15]:
df = open_group_file()
df.head()

,fn,head,num_sent,rel,word,field
25,Z200410536.xml,lift,17,poss,hand,VG.Object
27,Z200410536.xml,acts,17,nsubj,hand,VG.Object
94,Z300448748.xml,subdues,2,dobj,people,VG.Human
121,Z300448748.xml,is,13,nsubj,hand,VG.Object
130,Z300448748.xml,set,17,dobj,servant,VG.Human


In [16]:
i,row=df.iterrows().next()
row['rel']
#list(df.to_dict('records'))

u'poss'

In [17]:
def make_crosstabs(df,lim_cols=2000,row_sum_min=10,syntax=False,rel_word=REL_WORD):
    if rel_word:
        df['rel_head']=[unicode(row.get('rel',''))+'_'+unicode(row.get('head','')) for row in df.to_dict('records')]
        dfc=pd.crosstab(df['word'],df['rel_head'])
    else:
        left=pd.crosstab(df['word'], df['head'])
        right=pd.crosstab(df['word'], df['rel'])
        dfc=left.join(right,rsuffix='_rel')

    dfc=dfc.loc[dfc.sum(axis=1)>row_sum_min]
    cols=list(dfc.sum(axis=0).nlargest(lim_cols).index)
    dfc=dfc[cols]
    ## add field
    dfc['_field']=[word2field.get(w,'') for w in dfc.index]
    return dfc

In [18]:
dfc=make_crosstabs(df)
dfc.head()

rel_head,nsubj_is,nsubj_was,nsubj_be,dobj_have,dobj_give,dobj_make,dobj_had,dobj_take,dobj_see,nsubj_did,...,nsubj_chased,nsubj_check,nsubj_cleanse,nsubj_clown,nsubj_cold,nsubj_command,nsubj_commands,nsubj_commend,nsubj_commends,_field
word,,,,,,,,,,,,,,,,,,,,,
acquaintance,0,0,2,2,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,VG.Human
anchor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,VG.Object
arm,3,1,2,0,0,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,VG.Object
arrow,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,VG.Object
art,23,8,10,9,3,1,6,1,1,4,...,0,0,0,0,0,0,0,0,0,VG.Object


In [22]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut

def classify(X,y):
    loo=LeaveOneOut()
    correct=[]
    for train_index, test_index in loo.split(X):
        clf = LogisticRegression(C=0.001)
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        clf.fit(X_train,y_train)
        predictions=clf.predict(X_test)
        correct+=[int(y_test[0]==predictions[0])]
    return np.mean(correct)

from scipy.stats import zscore
def do_classification(df,target='_field',numruns=30,numsample=50,replace=False):
    target_types=set(list(df[target]))
    
    objects=[]
    for tt1 in target_types:
        for tt2 in target_types:
            if tt2<=tt1: continue
            for nr in range(numruns):
                objects+=[(tt1,tt2,nr)]
    
    import random
    random.shuffle(objects)
    numobj=len(objects)
    for i,(tt1,tt2,nr) in enumerate(objects):
        print i,numobj,tt1,tt2,nr,
        dfs=[df.loc[df[target]==tt] for tt in [tt1,tt2]]
        lens=[len(_df.index) for _df in dfs]
        minlen=min(lens)
        print lens,minlen,
        ns=numsample if numsample else minlen
        print ns,
        try:
            dfs_sample=[_df.sample(ns,replace=replace) for _df in dfs]
        except ValueError:
            print "!!"
            continue
        
        ndf=pd.concat(dfs_sample)
        Xdf=ndf.select_dtypes('number').apply(zscore).dropna(1)
        y=np.array([word2field[w] for w in Xdf.index])
        X=Xdf.values
        
        acc=classify(X,y)
        print acc
        odx={'class1':tt1,'class2':tt2,'accuracy':acc,'numruns':numruns,'numrun':nr,'numsample':numsample}
        #print odx
        yield odx

In [23]:
def classify_group_file(fn='data_booknlp/data_by_quarter_century/1600-1624.txt',
                        ofn='data_booknlp/data_by_quarter_century_modeled/1600-1624.txt',
                        only_rels=rels,only_fields=True,
                        numruns=30,lim_cols=2000):
    df=open_group_file(fn=fn,only_rels=only_rels,only_fields=only_fields)
    df_tabs=make_crosstabs(df,lim_cols=lim_cols)
    for odx in do_classification(df_tabs,numruns=numruns):
        odx['fn']=os.path.basename(fn)
        odx['period_int']=fn.split('-')[0]
        yield odx

In [25]:
classify_group_file().next()

0 90 VG.Human VG.Object 11 [84, 274] 84 50 0.72


{'accuracy': 0.72,
 'class1': 'VG.Human',
 'class2': 'VG.Object',
 'fn': '1600-1624.txt',
 'numrun': 11,
 'numruns': 30,
 'numsample': 50,
 'period_int': 'data_booknlp/data_by_quarter_century/1600'}

In [26]:
def do_classify_group_file(fn):
    return list(classify_group_file(fn))

def classify_all(idir='data_booknlp/data_by_quarter_century',
                 odir='data_booknlp/data_by_quarter_century_modeled',lim_cols=2000):
    import multiprocessing as mp
    pool=mp.Pool()
    filenames = [os.path.join(idir,fn) for fn in os.listdir(idir) if fn.endswith('.txt')]
    for old in pool.imap_unordered(do_classify_group_file, filenames):
        for odx in old:
            yield odx

In [28]:
#tools.writegen('data_booknlp/data.classification_results.txt', classify_all)
# last run, V2 (with normalization), 2/4/19 15:09

#tools.writegen('data_booknlp/data.classification_results.txt', classify_all)
# last run, V3 (with rel_word), 2/4/19 16:15

### Results

* Accuracy looks ok? These are all binary classification problems. Thirty times (numruns) per quarter-century, predicting between 50 words (numsamples) of class1 and 50 words of class2, *without* standardization:

<center><img src="images/Accuracy for predicting humananimalobject, 1600-2000.png" width="500"></center>

* Accuracy gets worse when we turn on standardization (Z-score) for features. Why?

<center><img src="images/Accuracy for predicting humananimalobject, 1600-2000.V2 with standardization.png" width="500"></center>

* Accuracy marginally better when "rel_head" (eg nsubj_knows) is used and not just "head" (eg knows). (Z-scores used)

<center><img src="images/Accuracy for predicting humananimalobject, 1600-2000.V3 with rel_word.png" width="500"></center>

* Median accuracy at the end of the day: **75%**, Human-vs-Object. Is that good enough to justify the next step?

<center><img src="images/Median accuracy rates per classification task (across all runs of all periods).png" width="800"></center>

## Next step: estimating humanness

Here we applying this machine-learnt model (separating human and objects) to estimate the 'humanness' of all other words in the data. We're not concerned with whether these estimations are “right,” per se, but more in the pattern of their wrongness: the word “nature” is not a person, but is there a history to its person-likeness? Is “nature” ever... dare I say... anthropomorphic: human-*like*, according to the model?


### Saving cross-tabs (2000MFW)

In [65]:
# Save crosstabs
def save_crosstabs(fn='data_booknlp/data_by_quarter_century/1600-1624.txt',
                          odir='data_booknlp/data_by_quarter_century__crosstabs',
                    only_rels=rels,field1='VG.Human',field2='VG.Object',
                    row_sum_min=10,lim_cols=2000):
    df=pd.read_csv(fn,sep='\t',encoding='utf-8',quoting=3,error_bad_lines=False)
    
    if only_rels: df=df.loc[df.rel.isin(rels)]
    df['field']=[word2field.get(w,'') for w in df.word]
    #df=df.loc[df.field.isin({field1,field2})]
    
    ## make crosstabs
    print '>> crosstabbing head counts',tools.now()
    counts_head=pd.crosstab(df['word'], df['head'])
    print '>> crosstabbing rel counts',tools.now()
    counts_rel=pd.crosstab(df['word'], df['rel'])
    print '>> joining tables',tools.now()
    df_tabs=counts_head.join(counts_rel,rsuffix='_rel')
    print '>> filtering by row_sum_min',tools.now()
    if row_sum_min: df_tabs=df_tabs.loc[df_tabs.sum(axis=1)>row_sum_min]
    print '>> filtering by lim_cols',tools.now()
    if lim_cols:
        cols=list(df_tabs.sum(axis=0).nlargest(lim_cols).index)
        df_tabs=df_tabs[cols]
    ## add field
    print '>> adding new column',tools.now()
    df_tabs['_field']=[word2field.get(w,'') for w in df_tabs.index]
    
    if not os.path.exists(odir): os.makedirs(odir)
    ofnfn=os.path.join(odir, os.path.basename(fn))
    print '>> saving',tools.now()
    df_tabs.to_csv(ofnfn,sep='\t',encoding='utf-8')
    print '>> saved:',ofnfn,tools.now()

In [72]:
def save_all_crosstabs(idir='data_booknlp/data_by_quarter_century/'):
    ifiles = [os.path.join(idir,ifn) for ifn in os.listdir(idir) if ifn.endswith('.txt')]
    tools.crunch(ifiles, save_crosstabs)    

In [74]:
#save_all_crosstabs()

#### Machine learning

In [303]:
import pandas as pd
import math
def classify_from_crosstabs(fn='data_booknlp/data_by_quarter_century__crosstabs/1900-1924.txt',
                    odir='data_booknlp/data_by_quarter_century__model_results/',
                    field1='VG.Human',field2='VG.Object',target='_field',
                    lim_cols=1000):
    df_tabs=pd.read_csv(fn,sep='\t',encoding='utf-8',quoting=3,error_bad_lines=False).fillna('').set_index('word')
    
    word2field=dict(zip(df_tabs.index,df_tabs[target]))
    #df_tabs.drop(target,inplace)
    
    #if lim_cols:
    #    cols=df_tabs.select_dtypes('number').sum(axis=0).nlargest(lim_cols).index
    #    df_tabs=df_tabs[list(cols) + ['_field']]
    
    ### make test and training sets
    df_train = df_tabs.loc[df_tabs._field.isin({field1,field2})]
    df_test = df_tabs.loc[~df_tabs._field.isin({field1,field2})]
    
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(C=0.001)
    
    # Fit
    X_train=df_train.select_dtypes('number').apply(zscore).dropna(1) #.values
    y_train=[word2field[w] for w in X_train.index]
    clf.fit(X_train.values,y_train)
    
    # Save model results
    if not os.path.exists(odir): os.makedirs(odir)
    ofnfn=os.path.join(odir,os.path.basename(fn))
    
    df_feats = pd.DataFrame(clf.coef_).T
    df_feats.columns = ['coeff']
    df_feats['feat']=X_train.columns
    #df_feats.set_index('feat',inplace=True)
    df_feats.to_csv(ofnfn,sep='\t',encoding='utf-8')
    print '>> saved:',ofnfn
    
    # Predict
    X = df_tabs.select_dtypes('number').apply(zscore).dropna(1)
    X = X[X_train.columns]
    predictions=clf.predict_proba(X.values)
    n_dim = len(predictions[0])
    header=[('ProbClass%s' % (i+1)) for i in range(n_dim)]
    df_result=pd.DataFrame(predictions, columns=header)
    df_result['word']=df_tabs.index
    #df_result=df_result.set_index('word')
    return df_result

In [304]:
#classify_from_crosstabs()

In [305]:
def classify_all_from_crosstabs(idir='data_booknlp/data_by_quarter_century__crosstabs'):
    paths = [os.path.join(idir,fn) for fn in sorted(os.listdir(idir)) if fn.endswith('.txt')]
    for path in paths:
        print '>>',path
        df_result=classify_from_crosstabs(path)
        ld_result = df_result.to_dict('records')
        for dx in ld_result:
            dx['fn']=os.path.basename(path)
            dx['period']=dx['fn'].split('-')[0]
            yield dx    

In [306]:
#classify_all_from_crosstabs().next()

In [337]:
#tools.writegen('data_booknlp/data.classification_probabilities_by_word_by_period.txt', classify_all_from_crosstabs)
# last run, V2 (with standardization), 2/4/2019 in the morning

#### Results

* Meaningfulness of results seems to have improved with the turn to standardization. Here are the results without standardization:

<center><img src="images/Probability of being a human V1.png" width="500"></center>

* Here are the results with standardization:

<center><img src="images/Probability of being a human V2.png" width="500"></center>

## To do (2/4)

* Make a better worddb! I want less a million vectors, and more semantic fields (column "VG" should have "Human", "Object", etc). This way I can remove the VG.Human/VG.Object's from the results. (Or I could switch "X" to "X_test" above.)
* Investigate feature loadings. What predicts humanness? Maybe switch features to "rel_word". More meaningful that way.
* Can we use these features to classify moments of personification 'in real time', i.e. in the text?
* ???